# Steel City Steelworks, Part Deux

Steel City Steelworks has steel mills in three major US cities producing different amounts of steel each week to supply manufacturing plants in four other cities with different amounts of weekly demand.  The table below lists steel supplies in tons for each mill, steel demands in tons for each plant, and per-ton shipping costs for sending steel from each supply mill to each plant.

| From/To | Detroit | St. Louis | Chicago | Norfolk | Supply |
| :---: | :---: | :---: | :---: | :---: | :---: |
Birmingham | 14 | 9  | 16 | 18 | 150 |
Gary | 11 |  8 | 7 | 16 | 210 |
Pittsburgh |  16 | 12 | 10 | 22 | 320 |
Demand |  130 | 70 | 180 | 240 |  |

**Goal:** Determine a plan for transporting steel from mills to plants that satisfies all demand while minimizing total transportation cost.

This time, we will model this as a balanced Minimum Cost Network Flow Problem; that is, our model will take the form:

\begin{align*}
\underset{\mathbf{x} \in \mathbb{R}^{|\mathcal{E}|}}{\min} \quad & \mathbf{c}^\top \mathbf{x} \\
\text{s.t.} \quad \ \ &\mathbf{A}\mathbf{x} = \mathbf{b} \\
&\mathbf{p} \leq \mathbf{x} \leq \mathbf{q}
\end{align*}

where $\mathbf{A}$ is the incidence matrix of the graph.

To make this happen, we need to introduce a "dummy" demand node with a demand of 60 to balance out the problem.

In [ ]:
#Problem Data

num_mills = 3
num_plants = 5  # adding one more "dummy" plant
num_edges = num_mills * num_plants

#Define the incidence matrix A for our fully connected network
#Edges go from the mills (first 3 rows) to the plants (last 5 rows)
#Edge 1 is column 1, edge 2 is column 2, etc.
A = [1  1  1  1  1  0  0  0  0  0  0  0  0  0  0
     0  0  0  0  0  1  1  1  1  1  0  0  0  0  0
     0  0  0  0  0  0  0  0  0  0  1  1  1  1  1
    -1  0  0  0  0 -1  0  0  0  0 -1  0  0  0  0
     0 -1  0  0  0  0 -1  0  0  0  0 -1  0  0  0
     0  0 -1  0  0  0  0 -1  0  0  0  0 -1  0  0
     0  0  0 -1  0  0  0  0 -1  0  0  0  0 -1  0
     0  0  0 0  -1  0  0  0  0 -1  0  0  0  0 -1];

#Supplies (first 3 entries) + real demands (next 4 entries) + dummy demand (last entry)
b = [150, 210, 320, -130, -70, -180, -240, -60]

#Same costs as before, but vectorized and with costs of 0 assigned to edges going to dummy plant
costs = [14 9 16 18 0 11 8 7 16 0 16 12 10 22 0];

A

In [ ]:
#JuMP Model

using JuMP, HiGHS

steelcity2 = Model(HiGHS.Optimizer)

@variable(steelcity2, x[1:num_edges] >= 0)
@constraint(steelcity2, supplyanddemand, A*x .== b)

@objective(steelcity2, Min, sum(costs[i]*x[i] for i in 1:num_edges))

print(steelcity2)

In [ ]:
optimize!(steelcity2)

In [ ]:
@show objective_value(steelcity2)
@show value.(x)